In [19]:
import pandas as pd
import os
import numpy as np
import cv2

In [55]:
# File name for the metadata to use in app.py
METADATA_FILE = 'labels.csv'
# ordering of columns in BOTH this file and app.py
YOLO_ORDER = ['label', 'x', 'y', 'w', 'h', 'image']
APP_ORDER = ['label', 'xmin', 'ymin', 'xmax', 'ymax', 'image']

In [56]:
# list of image names
image_files = [x for x in os.listdir('train_data/obj') if x[-4:] == '.jpg']
label_files = [x for x in os.listdir('train_data/obj') if x[-4:] == '.txt']

In [57]:
# Add one row for each label
# For each image
temp = []
for file in label_files:
    image_file = file[:-3] + 'jpg'
    try:
        with open('train_data/obj/' + file, encoding='utf-8') as f:
            for line in f.readlines():
                temp.append([item for item in line.strip('\n').split(' ')] + [image_file])
    except:
        continue
metadata = pd.DataFrame(temp, columns=YOLO_ORDER)
print('metadata created')

metadata created


In [58]:
metadata

,label,x,y,w,h,image
0,0,0.5021428571428571,0.5036101083032491,0.9814285714285714,0.9783393501805054,pos_156.jpg
1,1,0.500625,0.5034246575342466,0.94875,0.9598825831702544,pos_630.jpg
2,1,0.496,0.5188,0.7248,0.9352,pos_624.jpg
3,0,0.45859375,0.03611111111111111,0.0640625,0.06944444444444445,pos_1809.jpg
4,1,0.459765625,0.27152777777777776,0.11953125,0.2791666666666667,pos_1809.jpg
...,...,...,...,...,...,...
4724,0,0.414453125,0.11805555555555555,0.09921875,0.125,pos_1810.jpg
4725,1,0.478125,0.4375,0.46041666666666664,0.6375,pos_629.jpg
4726,1,0.443359375,0.4041666666666667,0.11484375,0.23333333333333334,pos_1804.jpg
4727,1,0.698046875,0.36180555555555555,0.08984375,0.2013888888888889,pos_1804.jpg


In [59]:
# Convert integer category label to object name string
metadata.loc[metadata['label'] == '0', 'label'] = 'Hard Hat'
metadata.loc[metadata['label'] == '1', 'label'] = 'Safety Vest'

In [60]:
# Convert string coordinates to float coordinates
metadata['x'] = metadata['x'].astype(float)
metadata['y'] = metadata['y'].astype(float)
metadata['w'] = metadata['w'].astype(float)
metadata['h'] = metadata['h'].astype(float)

In [61]:
for i in range(len(image_files)):
    image = cv2.imread('train_data/obj/' + image_files[i])
    h, w = image.shape[:2]
    metadata.loc[metadata['image'] == image_files[i], ['x', 'w']] *= w
    metadata.loc[metadata['image'] == image_files[i], ['y', 'h']] *= h

In [62]:
metadata

,label,x,y,w,h,image
0,Hard Hat,351.5,279.0,687.0,542.0,pos_156.jpg
1,Safety Vest,400.5,514.5,759.0,981.0,pos_630.jpg
2,Safety Vest,620.0,648.5,906.0,1169.0,pos_624.jpg
3,Hard Hat,587.0,26.0,82.0,50.0,pos_1809.jpg
4,Safety Vest,588.5,195.5,153.0,201.0,pos_1809.jpg
...,...,...,...,...,...,...
4724,Hard Hat,530.5,85.0,127.0,90.0,pos_1810.jpg
4725,Safety Vest,229.5,210.0,221.0,306.0,pos_629.jpg
4726,Safety Vest,567.5,291.0,147.0,168.0,pos_1804.jpg
4727,Safety Vest,893.5,260.5,115.0,145.0,pos_1804.jpg


In [63]:
metadata['xmin'] = metadata['x'] - metadata['w']/2
metadata['ymin'] = metadata['y'] - metadata['h']/2
metadata['xmax'] = metadata['x'] + metadata['w']/2
metadata['ymax'] = metadata['y'] + metadata['h']/2

In [64]:
# Convert float coordinates to int coordinates
metadata['xmin'] = metadata['xmin'].astype(int)
metadata['ymin'] = metadata['ymin'].astype(int)
metadata['xmax'] = metadata['xmax'].astype(int)
metadata['ymax'] = metadata['ymax'].astype(int)

In [65]:
# Drop unused columns
metadata = metadata[APP_ORDER]

In [68]:
# Check that coordinates fit within image size
metadata[metadata['image'] == 'pos_1003.jpg']

,label,xmin,ymin,xmax,ymax,image
408,Hard Hat,253,50,290,91,pos_1003.jpg
409,Hard Hat,34,50,80,79,pos_1003.jpg
410,Hard Hat,338,57,380,88,pos_1003.jpg
411,Hard Hat,403,21,452,58,pos_1003.jpg
412,Safety Vest,19,99,74,193,pos_1003.jpg
413,Safety Vest,213,99,300,193,pos_1003.jpg
414,Safety Vest,341,100,408,157,pos_1003.jpg
415,Safety Vest,407,67,473,154,pos_1003.jpg


In [66]:
# Stop "Run All" execution before saving file
assert False

AssertionError: 

In [40]:
# Save file
metadata.to_csv(METADATA_FILE, index=False)